In [82]:
import os
import numpy as np
import pandas as pd 
from mlxtend.frequent_patterns import apriori

In [108]:
basketData = pd.read_table("./market_basket.txt",delimiter="\t",header=0)

In [109]:
basketData.head(10)


,ID,Product
0,1,Peaches
1,2,Vegetable_Oil
2,2,Frozen_Corn
3,3,Plums
4,4,Pancake_Mix
5,5,Cheese
6,6,Cauliflower
7,7,2pct_Milk
8,8,98pct_Fat_Free_Hamburger
9,8,Potato_Chips


In [85]:
#basketData.describe
basketData.shape

(12935, 2)

In [86]:
cl=basketData['Product'].sort_values().unique()
ln=basketData['ID'].unique()

matrice=np.zeros((len(ln),len(cl)), int)
k=0
for i in ln:
  df=basketData[basketData["ID"]==i]
  a=0
  for j in cl:
    if df['Product'].str.contains(j).any():
      matrice[k][a]=1
    a=a+1
  k=k+1

df = pd.DataFrame(matrice, index=ln,columns=cl)



In [87]:
#5-tableau croisé avec la bib "crosstab"
CrossTab= pd.crosstab(basketData.ID,basketData.Product)

#6-Affichage des 30 premières transactions et des 3 premiers produits.
print(CrossTab.iloc[:30,:3])

Product  100_Watt_Lightbulb  2pct_Milk  40_Watt_Lightbulb
ID                                                       
1                         0          0                  0
2                         0          0                  0
3                         0          0                  0
4                         0          0                  0
5                         0          0                  0
6                         0          0                  0
7                         0          1                  0
8                         0          0                  0
9                         0          0                  0
10                        0          0                  0
11                        0          0                  0
12                        0          0                  0
13                        0          0                  0
14                        0          0                  0
15                        0          0                  0
16            

In [110]:
import itertools

def ExtractItemset(df, n):
    return list(itertools.combinations(df, n))

def CalculSupport (df,subsets,n):
  table=[]
  x=0
  for i in subsets:
    subset=df[list(i)]
    cnt=subset[subset.sum(axis='columns')==n].count()
    table.append(cnt[0])
    x=x+1
  return(table)

In [111]:
#7-Script de l'algo apriori

product_max=2 #product_max can be 3 and 4 but it saturates the memory
min_supp=0.025*len(ln)

for i in range(1,product_max+1):
  subset=ExtractItemset(cl,i)
  Support=CalculSupport(df,subset,i)
  newDF= pd.DataFrame(subset)
  sup=pd.DataFrame(Support)
  sup.columns=['Support']
  frame=[newDF,sup]
  concats=pd.concat(frame,axis=1)
  C=concats[concats["Support"]>min_supp]  
  C

In [90]:
def Rule(item):
  it=item[:-1]
  for i in range(1,product_max):
    List=ExtractItemset(it,i)
    PremisList=List[:-i]
    CclList=List[i:]
    PremisSupport=CalculSupport(df,PremisList,i)
    TotalSupport=CalculSupport(df,[it],product_max)
  return(PremisList,CclList,np.array(TotalSupport)/np.array(PremisSupport))

In [113]:
AprioriRes=[]
cnt=0
for i in C.index:
  cnt=cnt+1
  ligne=list(C.loc[i,:])
  Premis,Ccl,Confidence=Rule(ligne)
  print('Rule :',Premis,'',Ccl,' With confidence equals to ',Confidence)

Rule : [('2pct_Milk',)]  [('98pct_Fat_Free_Hamburger',)]  With confidence equals to  [0.34899329]
Rule : [('2pct_Milk',)]  [('Apples',)]  With confidence equals to  [0.30872483]
Rule : [('2pct_Milk',)]  [('Aspirin',)]  With confidence equals to  [0.31543624]
Rule : [('2pct_Milk',)]  [('Bananas',)]  With confidence equals to  [0.2885906]
Rule : [('2pct_Milk',)]  [('Bologna',)]  With confidence equals to  [0.26174497]
Rule : [('2pct_Milk',)]  [('Broccoli',)]  With confidence equals to  [0.28187919]
Rule : [('2pct_Milk',)]  [('Canned_Tuna',)]  With confidence equals to  [0.26845638]
Rule : [('2pct_Milk',)]  [('Cantaloupe',)]  With confidence equals to  [0.24832215]
Rule : [('2pct_Milk',)]  [('Chardonnay',)]  With confidence equals to  [0.29530201]
Rule : [('2pct_Milk',)]  [('Cheese',)]  With confidence equals to  [0.66442953]
Rule : [('2pct_Milk',)]  [('Chocolate_Bar',)]  With confidence equals to  [0.24161074]
Rule : [('2pct_Milk',)]  [('Cola',)]  With confidence equals to  [0.45637584]


In [92]:
#extraction des itemset freq avec la bib "mlxtend" 
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
!pip install apriori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
freq_itemsets = apriori(CrossTab,min_support=0.025,max_len=4,use_colnames=True)


In [95]:
print(freq_itemsets.columns)


Index(['support', 'itemsets'], dtype='object')


In [96]:
print(freq_itemsets.shape)


(603, 2)


In [97]:
#8-Afficher les 15 premiers itemsets
print(freq_itemsets.head(15))


     support                    itemsets
0   0.030147        (100_Watt_Lightbulb)
1   0.109559                 (2pct_Milk)
2   0.037500         (60_Watt_Lightbulb)
3   0.031618         (75_Watt_Lightbulb)
4   0.093382  (98pct_Fat_Free_Hamburger)
5   0.031618         (AA_Cell_Batteries)
6   0.025735    (Apple_Cinnamon_Waffles)
7   0.026471               (Apple_Drink)
8   0.031618          (Apple_Fruit_Roll)
9   0.032353                 (Apple_Jam)
10  0.033088               (Apple_Jelly)
11  0.032353               (Apple_Sauce)
12  0.053676                    (Apples)
13  0.066912                   (Aspirin)
14  0.027941               (Avocado_Dip)


In [98]:
#9-fonction fonction is_inclus() qui permet de vérifier si un sous-ensemble items est inclus dans l’ensemble x.
def is_inclus(x,items):
    return items.issubset(x)

In [99]:
#10-Afficher les itemsets comprenant le produit ‘Aspirin’
id = np.where(freq_itemsets.itemsets.apply (is_inclus,items={'Aspirin'}))

print(id)

(array([ 13, 208, 249, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281,
       282, 283, 284, 285, 286, 509, 510, 511, 552, 553, 554, 555, 556]),)


In [100]:
#11-Afficher les itemsets contenant Aspirin et Eggs
id = np.where(freq_itemsets.itemsets.apply (is_inclus,items={'Aspirin', 'Eggs'}))

print(id)
print(freq_itemsets.loc[id])

(array([274, 509, 552, 553]),)
      support                       itemsets
274  0.038235                (Eggs, Aspirin)
509  0.025735     (2pct_Milk, Eggs, Aspirin)
552  0.025000  (Eggs, Potato_Chips, Aspirin)
553  0.029412   (Eggs, White_Bread, Aspirin)


In [101]:
from mlxtend.frequent_patterns import association_rules

In [114]:
#12-Génération des règles à partir des itemsets fréquents
regles = association_rules(freq_itemsets,metric="confidence",min_threshold=0.75)

#type de l'objet renvoyé
tp = type(regles)
tp

pandas.core.frame.DataFrame

In [115]:
#13 Afficher les 5 premières règles. 
print(regles.iloc[:5,:])

                 antecedents    consequents  antecedent support  \
0       (Aspirin, 2pct_Milk)  (White_Bread)            0.034559   
1     (Bananas, White_Bread)    (2pct_Milk)            0.032353   
2       (Bananas, 2pct_Milk)  (White_Bread)            0.031618   
3        (Wheat_Bread, Cola)    (2pct_Milk)            0.032353   
4  (Popcorn_Salt, 2pct_Milk)         (Eggs)            0.033088   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.119118  0.027206    0.787234  6.608878  0.023089    4.140147  
1            0.109559  0.025735    0.795455  7.260525  0.022191    4.353268  
2            0.119118  0.025735    0.813953  6.833190  0.021969    4.734743  
3            0.109559  0.025735    0.795455  7.260525  0.022191    4.353268  
4            0.122794  0.027206    0.822222  6.695941  0.023143    4.934283  


In [105]:
#règles en restreignant l'affichage à qqs colonnes
myRegles = regles.loc[:,['antecedents','consequents','lift']]
print(myRegles.shape)

(50, 3)


In [117]:
#14-Filtrer les règles en affichant celles qui présentent un LIFT supérieur ou égal à 7.
res = myRegles[myRegles['lift'].ge(7.0)]
res

,antecedents,consequents,lift
1,"(Bananas, White_Bread)",(2pct_Milk),7.260525
3,"(Wheat_Bread, Cola)",(2pct_Milk),7.260525
8,"(Wheat_Bread, Onions)",(2pct_Milk),7.573897
10,"(Wheat_Bread, Potatoes)",(2pct_Milk),7.053081
13,"(Wheat_Bread, Toothpaste)",(2pct_Milk),7.379694
16,"(Hamburger_Buns, White_Bread)",(98pct_Fat_Free_Hamburger),8.202379
17,"(98pct_Fat_Free_Hamburger, Wheat_Bread)",(White_Bread),7.555556
29,"(Hot_Dog_Buns, Sweet_Relish)",(Hot_Dogs),9.031422
35,"(Toilet_Paper, Potatoes)",(White_Bread),7.318772
37,"(Toilet_Paper, Toothpaste)",(White_Bread),7.345679


In [119]:
#15-Filtrer les règles menant au conséquent {‘2pct_milk’}
res = myRegles[myRegles['consequents'].eq({'2pct_Milk'})]
res


,antecedents,consequents,lift
1,"(Bananas, White_Bread)",(2pct_Milk),7.260525
3,"(Wheat_Bread, Cola)",(2pct_Milk),7.260525
7,"(Eggs, Wheat_Bread)",(2pct_Milk),6.970104
8,"(Wheat_Bread, Onions)",(2pct_Milk),7.573897
9,"(Potato_Chips, Toothpaste)",(2pct_Milk),6.979866
10,"(Wheat_Bread, Potatoes)",(2pct_Milk),7.053081
13,"(Wheat_Bread, Toothpaste)",(2pct_Milk),7.379694
40,"(Eggs, Potato_Chips, White_Bread)",(2pct_Milk),7.143274
43,"(Eggs, Toothpaste, White_Bread)",(2pct_Milk),7.260525
46,"(Potato_Chips, Toothpaste, White_Bread)",(2pct_Milk),7.569160
